In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

# Exploring the data
##  Loading Data
Read the Iowa data file into a Pandas DataFrame called `home_data`.

In [ ]:
import pandas as pd

# Path of the file to read
iowa_train_path = '../input/train.csv'
iowa_test_path = '../input/test.csv'

# Read csv
home_data = pd.read_csv(iowa_train_path)
home_data_test = pd.read_csv(iowa_test_path)

# Look up dataset
home_data.head()

##  Review The Data

In [ ]:
# Print summary statistics
home_data.describe()

##  Assigning Prediction Target
Select the target variable, which corresponds to the sales price. Saving this to a new variable called `y`. 

In [ ]:
# print the list of columns in the dataset
home_data.columns

In [ ]:
# predicted training set
y = home_data.SalePrice

##  Create X
 create a DataFrame called `X` holding the predictive features.

using just the following columns in the list :
    * LotArea
    * YearBuilt
    * 1stFlrSF
    * 2ndFlrSF
    * FullBath
    * BedroomAbvGr
    * TotRmsAbvGrd

In [ ]:
# Create the list of limited features below
feature_names = ['LotArea','YearBuilt','1stFlrSF','2ndFlrSF','FullBath','BedroomAbvGr','TotRmsAbvGrd']

# select data corresponding to features in feature_names for training set
X = home_data[feature_names]

In [ ]:
# print description or statistics from X
print(X.describe())

# print the top few lines
print(X.head())

# Machine Learning model
## Specify and Fit Model
Create a `DecisionTreeRegressor` and save it iowa_model. 

fit the model just created using the data in `X` and `y` saved above.

In [ ]:
from sklearn.tree import DecisionTreeRegressor
#specify the model; model used here is DecisionTree. 
iowa_model = DecisionTreeRegressor(random_state=1)

# Fit the model
iowa_model.fit(X,y)

In [ ]:
predictions = iowa_model.predict(X)
print(predictions)

## Make Predictions
Make predictions with the model's `predict` command using `X` as the data. Save the results to a variable called `predictions`.

In [ ]:
print(predictions[:5])
print(y[:5])

# Model Validation
## Split Your Data
Use the `train_test_split` function to split up your data.


In [ ]:
# Import the train_test_split function
from sklearn.model_selection import train_test_split

train_X, val_X, train_y, val_y = train_test_split(X,y,random_state=1)

## Specify and Fit the Model

Create a `DecisionTreeRegressor` model and fit it to the training data.

In [ ]:
# Specify the model
iowa_model = DecisionTreeRegressor(random_state=1)

# Fit iowa_model with the training data.
iowa_model.fit(train_X,train_y)

## Make Predictions with Validation data

In [ ]:
val_predictions = iowa_model.predict(val_X)

In [ ]:
# print the top few validation predictions
print(val_predictions[:5])
# print the top few actual prices from validation data
print(y.head())

## Calculate the Mean Absolute Error in Validation Data

In [ ]:
from sklearn.metrics import mean_absolute_error
val_mae = mean_absolute_error(val_y,val_predictions)

# uncomment following line to see the validation_mae
print(val_mae)

# Preventing underfitting and overfitting to find optimal solution
## Function to return the mean absolute error for each combination of leaf nodes of a decision tree

In [ ]:
def get_mae(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

## Compare Different Tree Sizes
a loop that tries the following values for *max_leaf_nodes* from a set of possible values.

Call the *get_mae* function on each value of max_leaf_nodes

In [ ]:
candidate_max_leaf_nodes = [5, 25, 50, 100, 250, 500]
# Write loop to find the ideal tree size from candidate_max_leaf_nodes
for leaf in candidate_max_leaf_nodes:
    my_mae = get_mae(leaf,train_X, val_X, train_y, val_y)
    print("leaf:",leaf,"MAE:",my_mae)
# Store the best value of max_leaf_nodes (it will be either 5, 25, 50, 100, 250 or 500)
best_tree_size = 100

## Fit Model Using All Data
it would be even more accurate by using all of the data and keeping that tree size which is best

In [ ]:
# Fit the model with best_tree_size.
final_model = DecisionTreeRegressor(max_leaf_nodes=100, random_state=1)

# fit the final model
final_model.fit(X, y)


## Use a Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Define the model. Set random_state to 1
rf_model = RandomForestRegressor(random_state=1)

# fit your model
rf_model.fit(train_X,train_y)

# Calculate the mean absolute error of your Random Forest model on the validation data
rf_val_mae = mean_absolute_error(rf_model.predict(val_X),val_y)

print("Validation MAE for Random Forest Model: {}".format(rf_val_mae))


# Handling missing values
## Analysing missing values in entire dataset (Testing purpose)
## Trial and error of learnt concepts starts here

In [ ]:
# X.replace('NA', np.NaN)

missing_val_count_by_column = (home_data.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])


### On analysing the above missing values, out of a dataset of 1461 training values, 'Alley','PoolQC','Fence' and 'MiscFeature' have many null values
### and hence can be dropped

In [ ]:
# creating a copy of original dataset
home_data_cpy = home_data.copy()

#calculating threshold and setting it to 80%
thresh = len(home_data_cpy) * .2

#removing column if contains 80% of values as NA
home_data_cpy.dropna(thresh = thresh, axis = 1, inplace = True)

missing_val_count_by_column = (home_data_cpy.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

## Using imputer to fill the null values

In [ ]:
# columns containing null values
null_col = [col for col in home_data_cpy
                   if home_data_cpy[col].isnull().any()]
print("Null values before imputation:",null_col)
null_float_data = home_data_cpy.loc[:,home_data_cpy.dtypes==float]

#import imputer class
from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer()
imp_col = my_imputer.fit_transform(null_float_data)

imp_col

## Trial and error of concepts ends here

## Differentiating categorical and numerical variables to perform One hot encoding on categorical variables

In [ ]:
train_pred = home_data_cpy.drop(['Id','SalePrice'],axis=1)

# low cardinality columns are those having less unique values
low_cardinality_col = [cname for cname in train_pred if train_pred[cname].nunique()<10 and train_pred[cname].dtype == "object"]
low_cardinality_col

## Determining the numerical columns
### One hot encoding can only be performed for categorical variables
### One hot encoding is performed so that the entire dataset is of same datatype, for the machine learning models to predict and also for performing imputation

In [ ]:
numerical_col = [cname for cname in train_pred if train_pred[cname].dtype in ['int64','float64']]
numerical_col

## Using One hot encoder to encode categorical variables

In [ ]:
# from sklearn.preprocessing import OneHotEncoder
# onehotencoder = OneHotEncoder(categorical_features = low_cardinality_col)
# train_X_sample = onehotencoder.fit_transform()
train_pred = pd.get_dummies(train_pred)
low_cardinality_col = [cname for cname in train_pred if train_pred[cname].nunique()<10 and train_pred[cname].dtype == "object"]
low_cardinality_col


In [ ]:
missing_val_count_by_column = (train_pred.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

## Previously inputation was not able to be performed due to the presence of categorical variables, now since they have been encoded, the null values can be imputed using Imputer

In [ ]:
my_imputer = SimpleImputer()
imp_col = my_imputer.fit_transform(train_pred)

## End of another trial and error of One hot encoding

# XGBoost Model
## Fitting XGBooset model to training set

In [ ]:
from xgboost import XGBRegressor

my_model = XGBRegressor()
# Add silent=True to avoid printing out updates with each cycle
my_model.fit(train_X, train_y, verbose=False)

In [ ]:
# make predictions
predictions = my_model.predict(val_X)

from sklearn.metrics import mean_absolute_error
print("Mean Absolute Error : " + str(mean_absolute_error(predictions, val_y)))

In [ ]:
my_model = XGBRegressor(n_estimators=1000)
my_model.fit(train_X, train_y, early_stopping_rounds=5, 
             eval_set=[(val_X, val_y)], verbose=False)

In [ ]:
# make predictions
predictions = my_model.predict(val_X)

from sklearn.metrics import mean_absolute_error
print("Mean Absolute Error : " + str(mean_absolute_error(predictions, val_y)))